In [ ]:

import pandas as pd
import numpy as np

# Load the data from the ERE.xlsx file
file_path = 'ERE.xlsx'  # Replace with the correct path to your file
data_ere = pd.read_excel(file_path, usecols=['playerName', 'x', 'y', 'outcome'], nrows=100)

# Check if necessary columns exist
if {'playerName', 'x', 'y', 'outcome'}.issubset(data_ere.columns):
    # Simulate target coordinates (x_end, y_end) for pass calculations
    data_ere['x_end'] = data_ere['x'] + np.random.uniform(-20, 20, len(data_ere))
    data_ere['y_end'] = data_ere['y'] + np.random.uniform(-10, 10, len(data_ere))

    # Calculate pass distance
    data_ere['pass_distance'] = np.sqrt(
        (data_ere['x_end'] - data_ere['x']) ** 2 +
        (data_ere['y_end'] - data_ere['y']) ** 2
    )

    # Calculate pass angle
    data_ere['pass_angle'] = np.abs(
        np.arctan2(data_ere['y_end'] - data_ere['y'],
                   data_ere['x_end'] - data_ere['x'])
    )

    # Calculate xPass
    data_ere['xPass'] = 1 - (
        0.02 * data_ere['pass_distance'] +
        0.1 * data_ere['pass_angle']
    )
    data_ere['xPass'] = data_ere['xPass'].clip(0, 1)  # Ensure probabilities are between 0 and 1

    # Calculate Pre-Action Danger using xPass
    data_ere['pre_action_danger'] = data_ere['xPass']

    # Simulate Post-Action Danger (reduced if defensive action is successful)
    data_ere['post_action_danger'] = data_ere['pre_action_danger'] * np.where(data_ere['outcome'] == 1, 0.5, 1.0)

    # Calculate xDef
    data_ere['xDef'] = data_ere['pre_action_danger'] - data_ere['post_action_danger']

    # Group by playerName and calculate totals for xPass and xDef
    player_totals = data_ere.groupby('playerName').agg(
        total_xPass=('xPass', 'sum'),
        total_xDef=('xDef', 'sum')
    ).reset_index()

    # Save the results
    player_totals.to_csv('player_totals.csv', index=False)
    print("Player totals saved as 'player_totals.csv'")
else:
    print("The necessary columns 'playerName', 'x', 'y', or 'outcome' are not present in the dataset.")
